In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx

# === 学生定着率と経済要因の相関分析 ===
# 目的: 公開統計（JASSO、学校基本調査など）のトレンドに基づき、
# 経済的困窮が退学リスクに与える構造的要因を可視化する。

# 日本語フォント対策（英語表記をデフォルトに設定）
plt.rcParams['font.family'] = 'sans-serif'

# ----------------------------------------------------------
# 1. 構造的因果モデル (Causal Path Diagram)
# ----------------------------------------------------------
# 経済的困窮がいかにして「学業不振（GPA低下）」につながるかのパス図
def create_causal_path_diagram():
    G = nx.DiGraph()
    
    # ノード（要因）の定義
    nodes = [
        ('Eco', 'Economic Hardship\n(Remittance↓ Cost↑)'), # 経済的困窮
        ('Work', 'Excessive Work\n(>20h/week)'),          # 過剰なアルバイト
        ('Time', 'Time Poverty\n(No Sleep/Study)'),       # 時間貧困
        ('Acad', 'Academic Failure\n(Low GPA)'),          # 学業不振
        ('Mental', 'Mental Health\nIssues'),              # メンタル不調
        ('ROI', 'Low ROI / Taipa\nJudgment'),             # 投資対効果の悪化
        ('Drop', 'Dropout\n(Outcome)')                    # 中退
    ]
    
    G.add_nodes_from([n[0] for n in nodes])
    
    # 因果関係（エッジ）の定義
    edges = [
        ('Eco', 'Work'), ('Work', 'Time'), 
        ('Time', 'Acad'), ('Time', 'Mental'),
        ('Acad', 'Drop'), ('Mental', 'Drop'),
        ('Eco', 'ROI'), ('ROI', 'Drop')
    ]
    G.add_edges_from(edges)
    
    pos = {
        'Eco': (0, 1), 'Work': (1, 1), 'Time': (2, 1),
        'Acad': (3, 1.5), 'Mental': (3, 0.5),
        'ROI': (2, 0), 'Drop': (4, 1)
    }
    
    plt.figure(figsize=(10, 5))
    nx.draw(G, pos, with_labels=False, node_size=3000, node_color='lightblue', edgecolors='black', arrowsize=20)
    
    # ラベルの描画
    labels = {n[0]: n[1] for n in nodes}
    nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold')
    
    plt.title('Structural Causal Model of Student Dropout', fontsize=14)
    plt.axis('off')
    plt.show()

# ----------------------------------------------------------
# 2. 労働時間とGPAの相関 (Correlation Simulation)
# ----------------------------------------------------------
# 先行研究（Benner, 2018等）に基づき、週20時間を閾値としたGPA低下モデルを生成
def create_work_gpa_correlation():
    np.random.seed(42)
    n_students = 200
    
    # 労働時間 (0〜35時間/週)
    work_hours = np.random.uniform(0, 35, n_students)
    
    # GPAシミュレーション (労働時間が長いほど低下する傾向 + ノイズ)
    gpa = 3.5 - 0.06 * work_hours + np.random.normal(0, 0.4, n_students)
    gpa = np.clip(gpa, 0, 4.0)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(work_hours, gpa, alpha=0.6, color='purple', edgecolors='w', s=50)
    
    # トレンドライン
    m, b = np.polyfit(work_hours, gpa, 1)
    plt.plot(work_hours, m*work_hours + b, color='red', linewidth=2, label='Trend: GPA drops as Work increases')
    
    # 閾値ライン（週20時間：学修への影響が出始めるライン）
    plt.axvline(x=20, color='gray', linestyle='--', label='Threshold (20h/week)')
    
    plt.title('Correlation between Part-time Work Hours and GPA', fontsize=14)
    plt.xlabel('Weekly Work Hours', fontsize=12)
    plt.ylabel('GPA (Grade Point Average)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# 実行
create_causal_path_diagram()
create_work_gpa_correlation()